In [ ]:
#!/usr/bin/env python3
"""
GradeBook Analyzer
Author: Devansh Kaushik
Date: November 2025
Description: A CLI-based GradeBook Analyzer for analyzing and reporting student grades.
"""

import csv
import os
import sys
from collections import Counter
from statistics import mean, median
from typing import Dict, Tuple, List


PASS_MARK = 40.0


def print_welcome() -> None:
    print("\n===============================")
    print("   Welcome to GradeBook Analyzer")
    print("===============================\n")
    print("1. Enter student data manually")
    print("2. Load data from CSV file")
    print("3. Exit\n")


def get_data_manual() -> Dict[str, float]:
    marks: Dict[str, float] = {}
    while True:
        try:
            n_str = input("Enter number of students: ").strip()
            if n_str == "":
                print("Input cannot be empty. Enter 0 to cancel.")
                continue
            n = int(n_str)
            if n < 0:
                print("Number of students cannot be negative.")
                continue
            break
        except ValueError:
            print("Please enter a valid integer for the number of students.")
        except KeyboardInterrupt:
            print("\nInterrupted. Returning to menu.")
            return {}

    for i in range(1, n + 1):
        try:
            original_name = input(f"Enter student #{i} name: ").strip()
        except KeyboardInterrupt:
            print("\nInterrupted. Stopping manual entry.")
            break
        if original_name == "":
            original_name = f"Unnamed Student {i}"
        name = original_name
        dup_index = 1
        while name in marks:
            name = f"{original_name} ({dup_index})"
            dup_index += 1

        while True:
            try:
                score_str = input(f"Enter marks for {original_name}: ").strip()
                if score_str == "":
                    print("Score cannot be empty. Please enter a number (e.g., 75.5).")
                    continue
                score = float(score_str)
                if score < 0:
                    print("Score cannot be negative. Try again.")
                    continue
                break
            except ValueError:
                print("Invalid score. Please enter a numeric value (e.g., 85 or 72.5).")
            except KeyboardInterrupt:
                print("\nInterrupted. Skipping this student.")
                score = 0.0
                break

        marks[name] = score

    return marks


def get_data_from_csv() -> Dict[str, float]:
    marks: Dict[str, float] = {}
    file_path = input("Enter CSV file path (e.g. data.csv): ").strip()
    if not file_path:
        print("No file path provided.")
        return marks

    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        return marks

    try:
        # read small sample to detect dialect and header presence
        with open(file_path, newline='', encoding='utf-8') as sample_file:
            sample = sample_file.read(4096)
            sample_file.seek(0)
            try:
                dialect = csv.Sniffer().sniff(sample)
            except Exception:
                dialect = csv.get_dialect('excel')
            try:
                has_header = csv.Sniffer().has_header(sample)
            except Exception:
                has_header = False

        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f, dialect)
            if has_header:
                next(reader, None)
            row_num = 0
            for row in reader:
                row_num += 1
                if not row or all(cell.strip() == "" for cell in row):
                    continue
                if len(row) < 2:
                    print(f"Skipping row {row_num}: expected at least 2 columns (name, score).")
                    continue
                original_name = str(row[0]).strip() or f"Unnamed Student {row_num}"
                name = original_name
                dup_index = 1
                while name in marks:
                    name = f"{original_name} ({dup_index})"
                    dup_index += 1
                score_cell = row[1]
                try:
                    score = float(score_cell)
                    if score < 0:
                        print(f"Skipping row {row_num}: negative score {score_cell}.")
                        continue
                except ValueError:
                    print(f"Skipping row {row_num}: invalid numeric score '{score_cell}'.")
                    continue
                marks[name] = score

        if marks:
            print("CSV file loaded successfully!\n")
        else:
            print("CSV read completed but no valid student data was found.")
    except Exception as e:
        print(f"Error reading CSV file: {e}")

    return marks


def calculate_average(marks_dict: Dict[str, float]) -> float:
    return mean(marks_dict.values()) if marks_dict else 0.0


def calculate_median(marks_dict: Dict[str, float]) -> float:
    return median(marks_dict.values()) if marks_dict else 0.0


def find_max_score(marks_dict: Dict[str, float]) -> Tuple[str, float]:
    if not marks_dict:
        return ("", 0.0)
    name = max(marks_dict, key=lambda k: marks_dict[k])
    return name, marks_dict[name]


def find_min_score(marks_dict: Dict[str, float]) -> Tuple[str, float]:
    if not marks_dict:
        return ("", 0.0)
    name = min(marks_dict, key=lambda k: marks_dict[k])
    return name, marks_dict[name]


def assign_grades(marks_dict: Dict[str, float]) -> Dict[str, str]:
    grades: Dict[str, str] = {}
    for name, score in marks_dict.items():
        if score >= 90:
            grades[name] = 'A'
        elif score >= 80:
            grades[name] = 'B'
        elif score >= 70:
            grades[name] = 'C'
        elif score >= 60:
            grades[name] = 'D'
        else:
            grades[name] = 'F'
    return grades


def grade_distribution(grades: Dict[str, str]) -> Counter:
    return Counter(grades.values())


def pass_fail_lists(marks_dict: Dict[str, float]) -> Tuple[List[str], List[str]]:
    passed = [name for name, score in marks_dict.items() if score >= PASS_MARK]
    failed = [name for name, score in marks_dict.items() if score < PASS_MARK]
    return passed, failed


def print_results_table(marks_dict: Dict[str, float], grades: Dict[str, str]) -> None:
    sorted_items = sorted(marks_dict.items(), key=lambda kv: kv[1], reverse=True)
    name_col_width = max((len(name) for name, _ in sorted_items), default=4)
    print()
    print(f"{'Name':<{name_col_width}}  Marks   Grade")
    print("-" * (name_col_width + 15))
    for name, score in sorted_items:
        print(f"{name:<{name_col_width}}  {score:6.2f}   {grades.get(name, '-')}")
    print("-" * (name_col_width + 15))


def run_analysis(marks: Dict[str, float]) -> None:
    if not marks:
        print("No marks to analyze.")
        return

    avg = calculate_average(marks)
    med = calculate_median(marks)
    max_name, max_score = find_max_score(marks)
    min_name, min_score = find_min_score(marks)

    print(f"\nAverage Marks: {avg:.2f}")
    print(f"Median Marks: {med:.2f}")
    print(f"Highest Marks: {max_score:.2f} ({max_name})")
    print(f"Lowest Marks: {min_score:.2f} ({min_name})")

    grades = assign_grades(marks)
    dist = grade_distribution(grades)
    passed, failed = pass_fail_lists(marks)

    print_results_table(marks, grades)

    print("\nGrade Distribution:")
    for g in ['A', 'B', 'C', 'D', 'F']:
        print(f"{g}: {dist.get(g, 0)} students")

    total = len(marks)
    pass_count = len(passed)
    fail_count = len(failed)
    pass_percent = (pass_count / total * 100) if total else 0.0

    print(f"\nPassed Students ({pass_count}/{total}, {pass_percent:.1f}%): {', '.join(passed) if passed else 'None'}")
    print(f"Failed Students ({fail_count}/{total}): {', '.join(failed) if failed else 'None'}")


def main() -> None:
    try:
        while True:
            print_welcome()
            choice = input("Enter your choice (1-3): ").strip()
            if choice == '1':
                marks = get_data_manual()
            elif choice == '2':
                marks = get_data_from_csv()
            elif choice == '3':
                print("Exiting program. Goodbye!")
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
                continue

            if not marks:
                print("No data found or input cancelled. Returning to menu.\n")
                continue

            run_analysis(marks)

            try:
                again = input("\nDo you want to run another analysis? (y/n): ").strip().lower()
            except KeyboardInterrupt:
                print("\nInterrupted. Exiting.")
                break
            if again != 'y':
                print("Thank you for using GradeBook Analyzer!")
                break
    except KeyboardInterrupt:
        print("\nProgram interrupted by user. Exiting cleanly.")
        sys.exit(0)


if __name__ == "__main__":
    main()





   Welcome to GradeBook Analyzer

1. Enter student data manually
2. Load data from CSV file
3. Exit



Enter your choice (1-3):  2
Enter CSV file path (e.g. data.csv):  data.csv


File not found: data.csv
No data found or input cancelled. Returning to menu.


   Welcome to GradeBook Analyzer

1. Enter student data manually
2. Load data from CSV file
3. Exit

